In [1]:
import os

In [2]:
%pwd

'd:\\others\\NLP Projects\\Text-Summarization\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\others\\NLP Projects\\Text-Summarization'

3. Update entity (return type of a function)

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

4. Update the configuration manager in src config

In [6]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

5. Update the components (nothing but our dataization data, validation data, transformation these are the things)

In [8]:
import os
import urllib.request as request
import zipfile
from textSummerizer.logging import logger
from textSummerizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


6. Update the pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-03 18:28:35,437: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-03 18:28:35,440: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-03 18:28:35,442: INFO: common: created directory at: artifacts]
[2024-06-03 18:28:35,445: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-03 18:28:38,739: INFO: 2388645855: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8ACD:2EE6D5:4BF9DB:5CEA4F:665DB6F5
Accept-Ranges: bytes
Date: Mon, 03 Jun 2024 12:28:38 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1261-QPG
X-Cache: MI